In [2]:
import transformers
nlp = transformers.pipeline("conversational", 
                            model="microsoft/DialoGPT-medium")

C:\Users\shivansh\Chatbot\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
df=pd.read_excel('AttachmentMIT-Academic_Calendar_2023-24.-1 (2).xlsx')

In [4]:
df.drop([0,1],inplace=True)
df.columns=df.iloc[0]

In [5]:
df.drop(2,inplace=True)

In [6]:
df

2,NaN,July -23,Aug -23,Sept -23,Oct -23,Nov -23,Dec -23,NaN,Jan -24,Feb -24,Mar -24,Apr -24,May -24,Jun -24,July -24
3,NaN,1 S,1 T,1 F,1 SUNDAY,1 W Kannada Rajyotsava,1 F,NaN,1M,1 TH,1 F,Last Date to\nApply for\nBranch Change\nUtsav,1 W,1 S,1 M
4,NaN,2 SUNDAY,2 W,2 S,2 M Gandhi Jayanthi,NaN,Publishing list of\nS unsuccessful\nstudents i...,NaN,2T,NaN,NaN,NaN,Publishing list of\n2 TH unsuccessful\nstudent...,NaN,2 T
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 F,Applying for Branch\nChange starts,NaN,NaN,2 SUNDAY,NaN
6,NaN,NaN,NaN,NaN,NaN,2 TH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,3 M,3 TH,3 SUNDAY,NaN,NaN,NaN,NaN,"II, IV, VI Semester\nB. Tech./II Semester\nM.T...",NaN,NaN,NaN,NaN,NaN,3 W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,NaN,NaN,NaN,30S Mid-term Exam,NaN,NaN,NaN,NaN,30 T,NaN,30 S,NaN,30 TH,NaN,NaN
129,NaN,NaN,NaN,NaN,30M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30 T
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31 W,29 TH,NaN,30T,31 F,30 S,NaN
131,NaN,NaN,NaN,NaN,31 T,30TH,31SUNDAY,NaN,NaN,NaN,31 SUNDAY,NaN,NaN,NaN,31 W


In [7]:
import numpy as np

In [7]:
df2=pd.read_excel('chat_data.xlsx')

In [8]:
from datasets import load_dataset

In [9]:
raw_dataset = load_dataset('csv', data_files = 'chat_data.csv')

Generating train split: 110 examples [00:00, 612.83 examples/s]


In [10]:
split = raw_dataset['train'].train_test_split(test_size=0.3, seed=42)

In [12]:
from transformers import AutoTokenizer
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [13]:
def tokenize_fn(batch):
    return tokenizer(batch['questions'], truncation = True)

In [14]:
tokenized_dataset = split.map(tokenize_fn, batched = True)

Map: 100%|██████████| 33/33 [00:00<00:00, 535.76 examples/s]


In [15]:
vocab_size=20000
maxlen=200

In [16]:
import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(output_dir='training_dir',
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=10,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=64,
                                  )

In [19]:
from sklearn.metrics import f1_score,accuracy_score

In [20]:
def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis=-1)
    acc = np.mean(predictions == labels)
    f1 = f1_score(labels, predictions, average = 'micro')
    return {'accuracy': acc, 'f1_score': f1}

In [21]:
trainer = Trainer(model,
                  training_args,
                  train_dataset = tokenized_dataset["train"],
                  eval_dataset = tokenized_dataset["test"],
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,No log,0.148207,1.000000,1.000000
2,No log,0.045899,1.000000,1.000000
3,No log,0.015783,1.000000,1.000000
4,No log,0.007176,1.000000,1.000000
5,No log,0.003995,1.000000,1.000000
6,No log,0.002845,1.000000,1.000000
7,No log,0.002282,1.000000,1.000000
8,No log,0.001865,1.000000,1.000000
9,No log,0.001670,1.000000,1.000000
10,No log,0.001614,1.000000,1.000000


TrainOutput(global_step=50, training_loss=0.04661311149597168, metrics={'train_runtime': 310.4024, 'train_samples_per_second': 2.481, 'train_steps_per_second': 0.161, 'total_flos': 7176970701480.0, 'train_loss': 0.04661311149597168, 'epoch': 10.0})

In [28]:
from transformers import pipeline

In [25]:
ls training_dir

 Volume in drive C is OS
 Volume Serial Number is 54FE-CB04

 Directory of C:\Users\shivansh\Chatbot\training_dir

27-02-2024  22:53    <DIR>          .
27-02-2024  22:53    <DIR>          ..
27-02-2024  22:49    <DIR>          checkpoint-10
27-02-2024  22:50    <DIR>          checkpoint-15
27-02-2024  22:50    <DIR>          checkpoint-20
27-02-2024  22:51    <DIR>          checkpoint-25
27-02-2024  22:51    <DIR>          checkpoint-30
27-02-2024  22:52    <DIR>          checkpoint-35
27-02-2024  22:52    <DIR>          checkpoint-40
27-02-2024  22:52    <DIR>          checkpoint-45
27-02-2024  22:49    <DIR>          checkpoint-5
27-02-2024  22:53    <DIR>          checkpoint-50
27-02-2024  22:45    <DIR>          runs
               0 File(s)              0 bytes
              13 Dir(s)  31,707,021,312 bytes free


In [29]:
saved_model = pipeline('text-classification',
                       model = 'training_dir/checkpoint-50')

In [27]:
predictions = saved_model(split['test']['questions'])

In [28]:
predictions[:10]

[{'label': 'LABEL_0', 'score': 0.9969584941864014},
 {'label': 'LABEL_1', 'score': 0.9995785355567932},
 {'label': 'LABEL_1', 'score': 0.9995409250259399},
 {'label': 'LABEL_0', 'score': 0.996957540512085},
 {'label': 'LABEL_0', 'score': 0.9969717264175415},
 {'label': 'LABEL_1', 'score': 0.99956876039505},
 {'label': 'LABEL_1', 'score': 0.9995331764221191},
 {'label': 'LABEL_0', 'score': 0.9969909191131592},
 {'label': 'LABEL_1', 'score': 0.9995424747467041},
 {'label': 'LABEL_0', 'score': 0.9969700574874878}]

In [29]:
def get_label(d):
    return int(d['label'].split('_')[1])
predictions = [get_label(d) for d in predictions]

In [30]:
predictions

[0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1]

In [36]:
print("acc:",accuracy_score(split['test']['label'], predictions))

acc: 1.0


In [37]:
print("f1:",f1_score(split['test']['label'], predictions, average = 'macro'))

f1: 1.0


In [38]:
saved_model('when is the deepavali vacation ')[0]['label']

'LABEL_1'

In [132]:
import string
translator=str.maketrans(' ',string.punctuation,'')

ValueError: the first two maketrans arguments must have equal length

In [ ]:
from spellchecker import SpellChecker
spell=SpellChecker()
def spellcheck(txt):
    t=""
    for word in txt.split():
        w=spell.correction(word)
        if(w==None):
               w=word
        t+=w+" "
    return t.strip()

In [11]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shivansh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [166]:
def replace(txt):
    txt=txt.replace('midterm','mid term')
    txt=txt.replace('retest','re test')
    txt=txt.replace('make up','makeup')
    txt=txt.replace('vii','seventh')
    txt=txt.replace('iii','third')
    txt=txt.replace('ii','second')
    
    txt=txt.replace(' iv ',' fourth ')
    txt=txt.replace(' v ',' fifth ')
    txt=txt.replace(' vi ',' sixth ')
    
    txt=txt.replace(' i ',' first ')
    
    
    return txt

In [169]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
ls=WordNetLemmatizer()
def preprocess_text(txt):
    txt=txt.lower()
    txt=txt.translate(translator)
    txt=txt.replace('\n',' ')
    print(txt)
    txt=replace(txt)
    
    stop_words = set(stopwords.words('english'))
    tokens=[x for x in word_tokenize(txt) if x not in stop_words]
    
    t=""
    for i in tokens:
        t+=ls.lemmatize(i)+" "
    t=t.strip()
    return t
        

In [155]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')


In [156]:
saved_model('When mid term exams')[0]['label']

'LABEL_1'

In [157]:
def calc_sim(t1,t2):
    emb1=model.encode([t1])[0]
    emb2=model.encode([t2])[0]
    sim= cosine_similarity([emb1],[emb2])
    return sim

In [158]:
from sklearn.metrics.pairwise import cosine_similarity

In [159]:
nltk.download('words')
from nltk.corpus import words
eng_words=words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\shivansh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [160]:
def check_vocab(txt):
    tokens= word_tokenize(txt)
    for w in tokens:
        if w not in eng_words:
            return (w, False)
    return (w,True)

In [161]:
def direct_search(txt):
    for i in df.select_dtypes(include='object').columns:
            for j in df[i]:
                if type(j)!=float and len(j)>0:
                    j=preprocess_text(j)
                    if txt in j:
                        print(f'{i}:{j}')

In [162]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model2 = hub.load(module_url)
def calc_sim2(t1,t2):
    emb1=model2([t1])[0]
    emb2=model2([t2])[0]
    sim= cosine_similarity([emb1],[emb2])
    return sim

In [163]:
preprocess_text("When are my mid term exams?")

when are my mid term exams


'mid term exam'

In [172]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
while True:
    input_text=input('User: ')
    if input_text.lower()=='bye':
        print('GoodBye')
        break
    input_text=input_text.lower().translate(translator)
    #input_text=spellcheck(input_text)
    input_text=preprocess_text(input_text)
    label=saved_model(input_text)[0]['label']
    print(label)
    print(input_text)
    f=0
    if(label=='LABEL_1'):
        w,b=check_vocab(input_text)
        print(b)
        if  b!= True:
            direct_search(w)
        else:
            for i in df.select_dtypes(include='object').columns:
                for j in df[i]:
                    if type(j)!=float and len(j)>0:
                        j=preprocess_text(j)
                        sim=calc_sim(input_text,j)
                        
                        if sim>0.5:
                            
                            print(f'{i}:{j}')
                            print(sim)
                            f=1
                            break
                if f==1:
                    break
                        
                        
            
    
    else:
        print(nlp(transformers.Conversation(input_text), pad_token_id=50256))

User: when is my midterm?
when is my midterm
LABEL_1
mid term
True
 holiday only for students tentative schedule for midterm retest 08112023 to 25112023 subject to change as per mahe circular tentative schedule for midterm retest 08042024 to 25042024
1 s
July -23:1
[[0.67060506]]
User: bye
GoodBye


In [165]:
replace(' ii iv vi semester b tech ii semester mtech mca 3w class start viii semester b tech iv semester mca project work start')

' secondfourthvi semester b tech second semester mtech mca 3w class start seventhi semester b techfourthsemester mca project work start'